In [1]:
import spdcalc as spdc
import numpy as np

Set up the crystal with desired properties.

In [2]:
setup = spdc.SPDCSetup.from_dict({
    "crystal": "KDP_1",
    "signal_wavelength": 1550e-9,
    "crystal_theta": np.radians(51.765), "pm_type": "Type1_e_oo",
    "periodic_poling_enabled": False, "pump_wavelength": 775e-9,
    "crystal_length": 2000e-6, "pump_bandwidth": 5.35e-9
}, with_defaults=True)

Set the dimension and then use the spdcalc library to get the exact range of signal and ilder wavelengths that will be used so that results can be directly compared to those of spdcalc.org. 

In [3]:
dim = 200
ranges = spdc.plotting.calc_plot_config_for_jsi(setup, dim)
x=ranges.get_x_values()
y=ranges.get_y_values()

xmin = round(x[0]*1e9, 2)
xmax = round(x[len(x)-1]*1e9, 2)
ymin = round(y[0]*1e9, 2)
ymax = round(y[len(y)-1]*1e9, 2)

print('Minimum signal wavelength: ' + str(xmin) + 
       ' nm. Maximum signal wavelength: ' + str(xmax) + ' nm')
print('Minimum signal wavelength: ' + str(ymin) + 
      ' nm. Maximum signal wavelength: ' + str(ymax) + ' nm')

Minimum signal wavelength: 1467.1 nm. Maximum signal wavelength: 1632.9 nm
Minimum signal wavelength: 1475.11 nm. Maximum signal wavelength: 1642.84 nm


Use the built in function to calculate a two dimensional array of joint spectral intensity (JSI) values, and take the element-by-element square root of this array to get the joint spectral amplitude (JSA). 

In [4]:
jsi_data = spdc.plotting.plot_jsi(setup, ranges)
JSI = np.reshape(jsi_data, (dim, dim))
np.savetxt('JSI_data', JSI)

JSA = np.sqrt(JSI)

Next, use the numpy linear algebra library to calculate the singular value decomposition of the JSA matrix. This function returns a three-element tuple, the second of which is a one-dimensional array of Schmidt coefficients, $\lambda_i$. To ensure proper normalization, we impose the condition that

$$ \sum\limits_{i}^N \lambda_i^2 = 1. $$

In [5]:
a = np.linalg.svd(JSA)[1]
a_norm = a/np.linalg.norm(a)

Then we can calculate the single-photon purity, equal to 

$$ P = \sum\limits_{i}^N \lambda_i^4 $$

which can then be used to calculate the Schmidt number using 

$$ K = \frac{1}{P}. $$

In [6]:
purity = np.sum(a_norm**4)

schmidt_num = 1/purity
print(schmidt_num)

6.738711627285404
